In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime, timedelta

In [2]:
# import data
indeed_df = pd.read_csv(r"C:\Users\bosst\Data Science Job Market Research\Data Collection\indeed_jobs.csv")
glassdoor_df = pd.read_csv(r"C:\Users\bosst\Data Science Job Market Research\Data Collection\glassdoor_jobs.csv")

In [3]:
indeed_df.head(5)

,Unnamed: 0,date,job_title,company_name,company_reviews,company_location,pay,job_type,benefits,job_description
0,0,Posted\nPosted 1 day ago,Senior Data Analyst [BHJOB13022_12997],Ignite Technical Resources.,2 reviews,"Richmond, BC",NaN,NaN,NaN,"On behalf of our Healthcare client, Ignite Tec..."
1,1,Posted\nPosted 2 days ago,Data Scientist,University of British Columbia,598 reviews,"Vancouver, BC","$6,068.92–$8,724.25 a month",Full-time,NaN,Staff - Non Union\nJob Category\nM&P - AAPS\nJ...
2,2,Posted\nPosted 3 days ago,Senior Financial Data Analyst (Adaptive Insigh...,Central 1 Credit Union,41 reviews,"Vancouver, BC•Hybrid remote","$96,300 a year",Full-time,"['Paid time off', 'Tuition reimbursement']",Central 1 cooperatively empowers credit unions...
3,3,Posted\nPosted 3 days ago,"Lead Data Engineer, AI",Recruiting From Scratch,41 reviews,"Vancouver, BC•Remote","$160,000–$280,000 a year",Full-time,"['Dental care', 'Vision care']",This is for a client of Recruiting from Scratc...
4,4,Posted\nPosted 4 days ago,Junior Data Scientist,Theory and Practice,41 reviews,"Vancouver, BC•Hybrid remote","$160,000–$280,000 a year",Full-time,"['Dental care', 'Vision care']",Description\nThe Junior Data Scientist will jo...


In [4]:
# clean date 
def clean_date(list):
    if list[0] == "Posted\nPosted": 
        return (date.today() - timedelta(days=(int(list[1].replace('+',''))), hours=0))
    if list[0] == "Employer\nActive":
        return (date.today() - timedelta(days=(int(list[1].replace('+','')) + 30), hours=0))
indeed_df['date'] = indeed_df['date'].str.split(' ').apply(clean_date) 

In [5]:
def get_pay(list):
    if list is np.nan: 
        return list
    convert_dic = {'year' : 1,'month' : 12,'hour' : 2087}
    if len(list) == 2: return str(convert_dic[list[-1]] * float(list[0]))
    elif len(list) == 3: return str(convert_dic[list[-1]] * float(list[0])) + ' - ' +  str(convert_dic[list[-1]] * float(list[1]))
        

In [6]:
# format pay to anually pay range
indeed_df['pay'] = indeed_df['pay'].str.replace(" an","").str.replace("From ","").str.replace(" a","").str.replace( "[$,]" , "", regex = True).str.split("[ –]",regex = True).apply(get_pay)

In [7]:
glassdoor_df.head()

,posting_age,job_title,company_name,location,pay,job_description
0,2d,Senior Data Analyst [BHJOB13022_12997],Ignite Technical Resources.,Richmond,"$51,269/yr (est.)","On behalf of our Healthcare client, Ignite Tec..."
1,3d,Data Scientist,University of British Columbia,Vancouver,"$7,397/mo (est.)",Staff - Non Union\nJob Category\nM&P - AAPS\nJ...
2,3d,Senior Machine Learning Developer,Market Jar Media Inc,Vancouver,"$79,579/yr (est.)","One Team. One Goal.\nAt MarketJar, we are a te..."
3,4d,Senior Data Scientist,Spring Financial Inc.,Vancouver,"$125,000/yr (est.)","About us:\nSince 2014, Spring Financial has be..."
4,4d,Senior Financial Data Analyst (Adaptive Insigh...,Central 1 Credit Union,Vancouver,"$96,300/yr (est.)",Central 1 cooperatively empowers credit unions...


In [8]:
# clean date 
def clean_date_glassdoor(post_age):
    return date.today() - timedelta(days = int(post_age))
glassdoor_df['posting_age'] = glassdoor_df['posting_age'].str.replace("d","").str.replace("+","").apply(clean_date_glassdoor) 

In [9]:
# clean pay
def clean_pay_glassdoor(pay):
    if pay[-2:] == 'mo': return str(float(pay.replace("mo",""))*12)
    else: return pay
glassdoor_df['pay'] = glassdoor_df['pay'].str.replace("[$,/(est. )yr]","",regex = True).apply(clean_pay_glassdoor)

In [10]:
# select only needed data
job_posting_df = pd.concat([indeed_df[['job_title','date','company_name','company_location','pay','job_description']], glassdoor_df[['job_title','posting_age','company_name','location','pay','job_description']].rename(columns = {'posting_age':'date','location':'company_location'})], axis = 0, ignore_index=True)

In [18]:
# drop duplicate row with same company, job title and date post
job_posting_df = job_posting_df.drop_duplicates(subset=['job_title',  'company_name'], keep='first').reset_index().drop(['index'], axis = 1)

In [19]:
job_posting_df.to_csv("job_posting_combined.csv", index = False)

In [20]:
job_posting_df

,job_title,date,company_name,company_location,pay,job_description
0,Senior Data Analyst [BHJOB13022_12997],2023-11-27,Ignite Technical Resources.,"Richmond, BC",NaN,"On behalf of our Healthcare client, Ignite Tec..."
1,Data Scientist,2023-11-26,University of British Columbia,"Vancouver, BC",72827.04000000001 - 104691.0,Staff - Non Union\nJob Category\nM&P - AAPS\nJ...
2,Senior Financial Data Analyst (Adaptive Insigh...,2023-11-25,Central 1 Credit Union,"Vancouver, BC•Hybrid remote",96300.0,Central 1 cooperatively empowers credit unions...
3,"Lead Data Engineer, AI",2023-11-25,Recruiting From Scratch,"Vancouver, BC•Remote",160000.0 - 280000.0,This is for a client of Recruiting from Scratc...
4,Junior Data Scientist,2023-11-24,Theory and Practice,"Vancouver, BC•Hybrid remote",160000.0 - 280000.0,Description\nThe Junior Data Scientist will jo...
...,...,...,...,...,...,...
118,Data Scientist,2023-10-29,Lightspark,Vancouver,96730,Who We Are\nLightspark is an innovative cleant...
119,Machine Learning Engineer,2023-10-29,MobSquad,Vancouver,131100,"ABOUT MOBSQUAD\nWe are a well-funded, hyper-gr..."
120,ML Engineer/Data Engineer,2023-10-29,Railtown AI Technologies,Vancouver,90353,Location\nHybrid in Vancouver BC\nDepartment\n...
121,AI ML Engineer,2023-10-29,Railtown AI Technologies,Vancouver,107056,Location\nHybrid in Vancouver BC\nDepartment\n...
